# oa to ward overlaps

In [49]:
import os
import pandas as pd
import geopandas as gpd

In [50]:
# get the spatial dataset variables from the first notebook: 
# wards, oa, lsoa, msoa, lbth_oa21_lsoa21_msoa21_lad21, oa_ward_lookup, oa_centroids
%run "./0.0-lbth-spatial-datasets.ipynb"

In [51]:
# find there oas straddle ward boundaries
gdf = gpd.overlay(oa, wards, how='intersection')

## get intersecting oas/wards


In [52]:
import folium

In [53]:
# use the geopandas explore function to get an interactive map
f = wards.explore()


In [54]:
# add ward name to oa from lookup table (generated by pop. weighted centroid matching)
oas = oa.merge(oa_ward_lookup, left_on='OA21CD', right_on='oa21cd')

In [55]:
# add straddling oas' pop. weighted centroids to map
folium.GeoJson(
    oas[list(gdf.groupby("OA21CD").count().OBJECTID > 1)]
    .join(oa_centroids.set_index("oa21cd"), on="OA21CD", lsuffix="_poly")
    .drop("geometry_poly", axis=1)
    .to_json(),
    marker=folium.CircleMarker(
        radius=2,  # Radius in metres
        weight=0,  # outline weight
        fill_color="magenta",
        fill_opacity=1,
    ),
    name="centroids",
).add_to(f)


In [56]:
# add straddling oas to map

def style_function(feature):
    return {"fillOpacity": 0, "weight": 1, "color": "#eee"}

folium.GeoJson(
    oas[list(gdf.groupby("OA21CD").count().OBJECTID > 1)].to_json(),
    style_function=style_function,
    name="oas",
    tooltip=folium.GeoJsonTooltip(
        fields=["OA21CD",'ward_name'],
        style=(
            "background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"
        ),
        sticky=True,
    ),
).add_to(f)


In [57]:
# add layer switches
folium.LayerControl().add_to(f)

In [58]:
# preview the map (doesn't show in github)
# can preview online: https://nbviewer.org/github/data-hamlets/census-data/blob/main/notebooks/0.2-lbth-oa-ward-overlaps.ipynb
f

In [59]:
# save map for sharing
f.save('../reports/oa-ward-overlaps.html')